# Advanced RAG with additional information about quality pf embeddings

# Advanced RAG with embeddings evaluation

![image.png](attachment:bea2e4c6-9a32-4206-8866-32e61a5d5239.png)

#### Table of contents

1. [Install dependencies](#section-id1).
2. [Loading knowledge base](#section-id2).
3. [Recursive chunking](#section-id3).
4. [FAISS](#section-id4).
5. [Calinski–Harabasz index](#section-id5).
5. [RAGAS](#section-id6).

<a id='section-id1'></a>
### Install dependencies

In [ ]:
%pip install -q torch transformers transformers accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl pacmap datasets langchain-community ragatouille
%pip install fsspec==2023.6.0
%pip install --upgrade pandas
%pip install --upgrade typing_extensions
%pip install numpy==1.22
%pip install --upgrade h5py
%pip install protobuf==3.20.3
%pip install --upgrade transformers
!python3 -m pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [1]:
!pip install torch transformers transformers accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl pacmap datasets langchain-community ragatouille
!pip install --upgrade typing_extensions
!pip install --upgrade h5py

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 108.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [1]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
from datasets import Dataset
import matplotlib.pyplot as plt

<a id='section-id2'></a>
### Loading knowledge base

Data info:

TriviaqQA is a reading comprehension dataset containing over 650K question-answer-evidence triples. TriviaqQA includes 95K question-answer pairs authored by trivia enthusiasts and independently gathered evidence documents, six per question on average, that provide high quality distant supervision for answering the questions.

The data fields are the same among all splits.

rc
* question: a string feature.
* question_id: a string feature.
* question_source: a string feature.
* entity_pages: a dictionary feature containing:
* doc_source: a string feature.
* filename: a string feature.
* title: a string feature.
* wiki_context: a string feature.
* search_results: a dictionary feature containing:
* description: a string feature.
* filename: a string feature.
* rank: a int32 feature.
* title: a string feature.
* url: a string feature.
* search_context: a string feature.
* aliases: a list of string features.
* normalized_aliases: a list of string features.
* matched_wiki_entity_name: a string feature.
* normalized_matched_wiki_entity_name: a string feature.
* normalized_value: a string feature.
* type: a string feature.
* value: a string feature.

In [2]:
from datasets import load_dataset

ds = load_dataset("mandarjoshi/trivia_qa", "rc.wikipedia")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/26 [00:00<?, ?it/s]

In [3]:
ds_train = ds['train']
ds_test = ds['test']

In [4]:
def extract_wiki_context(json_str):
    return json_str['wiki_context'][0]

ds_train = ds_train.map(lambda x: {'wiki_context': extract_wiki_context(x['entity_pages'])})
ds_test = ds_test.map(lambda x: {'wiki_context': extract_wiki_context(x['entity_pages'])})

In [5]:
dd = ds_test.shuffle(seed=42).select(range(1000))

In [6]:
from langchain.docstore.document import Document as LangchainDocument

RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["wiki_context"])
    for doc in tqdm(dd)
]

  0%|          | 0/1000 [00:00<?, ?it/s]

<a id='section-id3'></a>
#### Recursive chunking

Recursive chunking breaks down the text into smaller parts step by step using a given list of separators sorted from the most important to the least important separator. If the first split doesn't give the right size or shape of chunks, the method repeats itself on the new chunks using a different separator. For instance with the list of separators ["\n\n", "\n", ".", ""]:

The method will first break down the document wherever there is a double line break "\n\n".
Resulting documents will be split again on simple line breaks "\n", then on sentence ends ".".
Finally, if some chunks are still too big, they will be split whenever they overflow the maximum size.
With this method, the global structure is well preserved, at the expense of getting slight variations in chunk size.

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# We use a hierarchical list of separators specifically tailored for splitting Markdown documents
# This list is taken from LangChain's MarkdownTextSplitter class
MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

In [8]:
from transformers import AutoTokenizer

EMBEDDING_MODEL_NAME = "thenlper/gte-small"


def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: Optional[str] = EMBEDDING_MODEL_NAME,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name, truncate_dim=64),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS,
    )

    docs_processed = []
    for doc in tqdm(knowledge_base, desc="docs_processed ready..."):
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in tqdm(docs_processed, desc="docs_processed_unique ready..."):
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique


docs_processed = split_documents(
    512,  # We choose a chunk size adapted to our model
    RAW_KNOWLEDGE_BASE,
    tokenizer_name=EMBEDDING_MODEL_NAME,
)

docs_processed ready...:   0%|          | 0/1000 [00:00<?, ?it/s]

docs_processed_unique ready...:   0%|          | 0/15670 [00:00<?, ?it/s]

<a id='section-id4'></a>
#### FAISS

FAISS (Facebook AI Similarity Search) is a library developed by Facebook AI that is designed for efficient similarity search and clustering of dense vectors. It enables fast and scalable nearest neighbor searches, particularly useful for large datasets, by employing various techniques such as quantization and indexing to optimize both memory usage and search speed.

In [9]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

In [10]:
KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
    docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

<a id='section-id5'></a>
### Calinski–Harabasz index

The Calinski–Harabasz index (CHI), also known as the Variance Ratio Criterion (VRC), is a metric for evaluating clustering algorithms

Given a data set of n points: {x1, ..., xn}, and the assignment of these points to k clusters: {C1, ..., Ck}, the Calinski–Harabasz (CH) Index is defined as the ratio of the between-cluster separation (BCSS) to the within-cluster dispersion (WCSS), normalized by their number of degrees of freedom

$$
CH = \frac{\text{BCSS/(k - 1)}}{\text{WCSS/(n - k)}}
$$

$$BCSS = \sum_{i=0}^k n_i ||c_i - c||^2$$

$$WCSS = \sum_{k=0}^K \sum_{x\in C_i}^k ||x_i - c_k||^2$$

In [11]:
faiss_index = KNOWLEDGE_VECTOR_DATABASE.index
vectors = faiss_index.reconstruct_n(0, faiss_index.ntotal)

In [12]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(vectors)

In [13]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import calinski_harabasz_score

In [14]:
X = [
    list(KNOWLEDGE_VECTOR_DATABASE.index.reconstruct_n(idx, 1)[0]) for idx in range(len(docs_processed))
]


In [15]:
k = 3 # Количество кластеров
minibatch_kmeans = MiniBatchKMeans(n_clusters=k, batch_size=100, random_state=0)
minibatch_kmeans.fit(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1934: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)


MiniBatchKMeans(batch_size=100, n_clusters=3, random_state=0)

In [16]:
print(calinski_harabasz_score(vectors, minibatch_kmeans.labels_))

406.89310194711123


In [17]:
labels = minibatch_kmeans.labels_

In [18]:
import pacmap
import numpy as np
import plotly.express as px

embedding_projector = pacmap.PaCMAP(n_components=2, n_neighbors=None, MN_ratio=0.5, FP_ratio=2.0, random_state=1)

embeddings_2d = [
    list(KNOWLEDGE_VECTOR_DATABASE.index.reconstruct_n(idx, 1)[0]) for idx in range(len(docs_processed))
]

# Fit the data (the index of transformed data corresponds to the index of the original data)
documents_projected = embedding_projector.fit_transform(np.array(embeddings_2d), init="pca")

/usr/local/lib/python3.10/dist-packages/pacmap/pacmap.py:822: UserWarning: Warning: random state is set to 1
  warnings.warn(f'Warning: random state is set to {_RANDOM_STATE}')


In [19]:
df = pd.DataFrame.from_dict(
    [
        {
            "x": documents_projected[i, 0],
            "y": documents_projected[i, 1],
            "source": minibatch_kmeans.labels_[i],
            "extract": docs_processed[i].page_content[:100] + "...",
            "symbol": "circle",
            "size_col": 4,
        }
        for i in range(len(docs_processed))
    ]
)


In [20]:

# Visualize the embedding
fig = px.scatter(
    df,
    x="x",
    y="y",
    color="source",
    size="size_col",
    symbol="symbol",
    width=1000,
    height=700,
)
fig.update_traces(
    marker=dict(opacity=1, line=dict(width=0, color="DarkSlateGrey")),
    selector=dict(mode="markers"),
)
fig.update_layout(
    legend_title_text="<b>Chunk source</b>",
    title="<b>2D Projection of Chunk Embeddings via PaCMAP</b>",
)
fig.show()

In [21]:
user_query = 'Who is Elbert'

In [22]:
retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=user_query, k=5)

In [23]:
embedded_docs = embedding_model.embed_documents([doc.page_content for doc in retrieved_docs[:5]])
found_indices = np.array(embedded_docs)

In [24]:
import numpy as np
from sklearn.metrics import pairwise_distances
X = vectors
sample_vectors = found_indices  # Векторы 5 текстов выборки

def dist(X, sample_vectors, labels):
  sample_vectors = embedding_model.embed_documents([doc.page_content for doc in retrieved_docs[:5]])
  # Находим уникальные кластеры
  unique_labels = np.unique(labels)

  # Вычисляем центры кластеров (например, средние векторы)
  cluster_centers = []
  for label in unique_labels:
      if label != -1:  # игнорируем шум
          cluster_data = X[labels == label]
          cluster_center = np.mean(cluster_data, axis=0)
          cluster_centers.append(cluster_center)

  cluster_centers = np.array(cluster_centers)

  # Теперь вычисляем расстояния от выборки до ближайших центров кластеров
  distances = pairwise_distances(sample_vectors, cluster_centers)

  # Находим минимальное расстояние для каждого текста в выборке
  nearest_distances = np.min(distances, axis=1)
  return nearest_distances


<a id='section-id6'></a>
### RAGAS evaluation

$
Faithfulness = \frac{\text{|Number of claims that can be inferred from given context|}}{\text{|Total number of claims in the generated answer|}}
$

* $ \text{context recall} = \frac{\text{|GT sentences that can be attributed to context|}}{\text{|Number of sentences in GT|}} $

* $Context Precision@k = \frac{\sum \text{precision@k}}{\text{total number of relevant items in the top K results}}$

* $Precision@k = \frac{\text {true positives@k}}{\text{(true positives@k + false positives@k)}}$

In [25]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

READER_MODEL_NAME = "VityaVitalich/Llama3.1-8b-instruct" # ЗАМЕНИТЬ НА ЛАМУ

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [61]:
## Поправить шаблон

prompt_in_chat_format_quat = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.
After each document you will receive a description of the complexity of this document in the format of floating point number. The more complex the document, the less useful it is for answering.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE_quat = tokenizer.apply_chat_template(
    prompt_in_chat_format_quat, tokenize=False, add_generation_prompt=True
)

In [27]:
# НЕ МЕНЯТЬ

prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)

In [28]:
from ragatouille import RAGPretrainedModel

RERANKER = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884



In [29]:
from transformers import Pipeline


def answer_with_rag_quat(
    question: str,
    llm: Pipeline,
    knowledge_index: FAISS,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 5,
) -> Tuple[str, List[LangchainDocument]]:
    # Gather documents with retriever
    # print("=> Retrieving documents...")
    relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]  # Keep only the text

    # Optionally rerank results
    if reranker:
        # print("=> Reranking documents...")
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]
    dist_val = dist(X, relevant_docs, labels)

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + relevant_docs[i] + 'Complexity:' + str(dist_val[i]) for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE_quat.format(question=question, context=context)

    # Redact an answer
    # print("=> Generating answer...")
    answer = llm(final_prompt)[0]["generated_text"]

    return answer, relevant_docs

In [30]:
def answer_with_rag(
    question: str,
    llm: Pipeline,
    knowledge_index: FAISS,
    reranker: Optional[RAGPretrainedModel] = None,
    num_retrieved_docs: int = 30,
    num_docs_final: int = 5,
) -> Tuple[str, List[LangchainDocument]]:
    # Gather documents with retriever
    # print("=> Retrieving documents...")
    relevant_docs = knowledge_index.similarity_search(query=question, k=num_retrieved_docs)
    relevant_docs = [doc.page_content for doc in relevant_docs]  # Keep only the text

    # Optionally rerank results
    if reranker:
        # print("=> Reranking documents...")
        relevant_docs = reranker.rerank(question, relevant_docs, k=num_docs_final)
        relevant_docs = [doc["content"] for doc in relevant_docs]

    relevant_docs = relevant_docs[:num_docs_final]

    # Build the final prompt
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(relevant_docs)])

    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Redact an answer
    # print("=> Generating answer...")
    answer = llm(final_prompt)[0]["generated_text"]

    return answer, relevant_docs

In [31]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging, GenerationConfig, AutoModel
from tqdm.notebook import tqdm
import torch
import pandas as pd
import json
from datasets import load_dataset
import re
from torch import nn

In [32]:
class RAGAS():
  def __init__(self, params):
    judge_name = params["judge_name"]
    embedder_name = params["embedder_name"]

    quant_config = BitsAndBytesConfig(load_in_4bit=True,
                                  bnb_4bit_compute_dtype=torch.float16,
                                  bnb_4bit_quant_type="nf4",
                                  bnb_4bit_use_double_quant=True)
    self.judge_model = AutoModelForCausalLM.from_pretrained(judge_name,
                                                      quantization_config=quant_config,
                                                      device_map='auto',
                                                      low_cpu_mem_usage=True, offload_state_dict=True)
    self.judge_model.eos_token = "<|eot_id|>"
    self.judge_model.pad_token = "<|eot_id|>"
    self.judge_model.padding_side = "left"
    self.judge_tokenizer = AutoTokenizer.from_pretrained(judge_name)

    self.embedder_model = AutoModel.from_pretrained(embedder_name)
    self.embedder_tokenizer = AutoTokenizer.from_pretrained(embedder_name)

    self.statements_extraction_prompt = """Given a question and answer, create one or more statements from each sentence in the given answer. Here is an example:
    question: "Who was Albert Einstein and what is he best known for?"
    answer: "He was a German-born theoretical physicist, widely acknowledged to be one of the greatest and most influential physicists of all time. He was best known for developing the theory of relativity, he also made important contributions to the development of the theory of quantum mechanics."
    statements: "Albert Einstein was a German-born theoretical physicist.",
                "Albert Einstein is recognized as one of the greatest and most influential physicists of all time.",
                "Albert Einstein was best known for developing the theory of relativity.",
                "Albert Einstein also made important contributions to the development of the theory of quantum mechanics."<|eot_id|>
    <|start_header_id|>
    question: {question}
    answer: {answer}
    <|end_header_id|>
    """
    self.statements_verification_prompt = """Your task is to judge the faithfulness of a series of statements based on a given context. For each statement you must return verdict as 1 if the statement can be directly inferred based on the context or 0 if the statement can not be directly inferred based on the context. Format the outputs in JSON." Here is an example:
    context: "John is a student at XYZ University. He is pursuing a degree in Computer Science. He is enrolled in several courses this semester, including Data Structures, Algorithms, and Database Management. John is a diligent student and spends a significant amount of time studying and completing assignments. He often stays late in the library to work on his projects."
    statements: "John is majoring in Biology.",
                "John is taking a course on Artificial Intelligence.",
                "John is a dedicated student.",
                "John has a part-time job."
    answer: {example}<|eot_id|>
    <|start_header_id|>
    context: {context}
    statements: {statements}
    <|end_header_id|>
    """
    verification_example = [
                    {
                        "statement": "John is majoring in Biology.",
                        "reason": "John's major is explicitly mentioned as Computer Science. There is no information suggesting he is majoring in Biology.",
                        "verdict": 0,
                    },
                    {
                        "statement": "John is taking a course on Artificial Intelligence.",
                        "reason": "The context mentions the courses John is currently enrolled in, and Artificial Intelligence is not mentioned. Therefore, it cannot be deduced that John is taking a course on AI.",
                        "verdict": 0,
                    },
                    {
                        "statement": "John is a dedicated student.",
                        "reason": "The context states that he spends a significant amount of time studying and completing assignments. Additionally, it mentions that he often stays late in the library to work on his projects, which implies dedication.",
                        "verdict": 1,
                    },
                    {
                        "statement": "John has a part-time job.",
                        "reason": "There is no information given in the context about John having a part-time job.",
                        "verdict": 0,
                    },
                ]
    self.verification_example = json.dumps(verification_example)
    self.answer_relevance_prompt = """Generate questions for the given answer. Here is an example:
    answer: "Albert Einstein was a German-born theoretical physicist, widely acknowledged to be one of the greatest and most influential physicists of all time. He was best known for developing the theory of relativity, he also made important contributions to the development of the theory of quantum mechanics."
    questions: "Where was Albert Einstein born?"
    "Who was Albert Einstein?"
    "What is Albert Einstein famous for?"
    <|eot_id|>
    <|start_header_id|>answer: {answer}<|end_header_id|>"""
    self.context_precision_prompt = """Given question, answer and context verify if the context was useful in arriving at the given answer. Give verdict as "1" if useful and "0" if not with json output. Here are some examples:
    Example 1
    "question": "What can you tell me about albert Albert Einstein?",
    "context": "Albert Einstein (14 March 1879 – 18 April 1955) was a German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time. Best known for developing the theory of relativity, he also made important contributions to quantum mechanics, and was thus a central figure in the revolutionary reshaping of the scientific understanding of nature that modern physics accomplished in the first decades of the twentieth century. His mass–energy equivalence formula E = mc2, which arises from relativity theory, has been called "the world's most famous equation". He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect", a pivotal step in the development of quantum theory. His work is also known for its influence on the philosophy of science. In a 1999 poll of 130 leading physicists worldwide by the British journal Physics World, Einstein was ranked the greatest physicist of all time. His intellectual achievements and originality have made Einstein synonymous with genius.",
    "answer": "Albert Einstein born in 14 March 1879 was German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time. He received the 1921 Nobel Prize in Physics for his services to theoretical physics. He published 4 papers in 1905. Einstein moved to Switzerland in 1895",
    "verification": {example_1}
    Example 2
    "question": "who won 2020 icc world cup?",
    "context": "The 2022 ICC Men's T20 World Cup, held from October 16 to November 13, 2022, in Australia, was the eighth edition of the tournament. Originally scheduled for 2020, it was postponed due to the COVID-19 pandemic. England emerged victorious, defeating Pakistan by five wickets in the final to clinch their second ICC Men's T20 World Cup title.",
    "answer": "England",
    "verification": {example_2}
    Example 3
    "question": "What is the tallest mountain in the world?",
    "context": "The Andes is the longest continental mountain range in the world, located in South America. It stretches across seven countries and features many of the highest peaks in the Western Hemisphere. The range is known for its diverse ecosystems, including the high-altitude Andean Plateau and the Amazon rainforest.",
    "answer": "Mount Everest.",
    "verification": {example_3}
    <|eot_id|>
    <|start_header_id|>
    "question": {question},
    "context": {context},
    "answer": {answer},<|end_header_id|>
    """
    self.cp_ex1 = json.dumps({"reason": "The provided context was indeed useful in arriving at the given answer. The context includes key information about Albert Einstein's life and contributions, which are reflected in the answer.", "verdict": 1})
    self.cp_ex2 = json.dumps({"reason": "the context was useful in clarifying the situation regarding the 2020 ICC World Cup and indicating that England was the winner of the tournament that was intended to be held in 2020 but actually took place in 2022.", "verdict": 1})
    self.cp_ex3 = json.dumps({"reason": "the provided context discusses the Andes mountain range, which, while impressive, does not include Mount Everest or directly relate to the question about the world's tallest mountain.", "verdict": 0})
    self.context_recall_prompt = """Given a context, and an answer, analyze each sentence in the answer and classify if the sentence can be attributed to the given context or not. Use only "Yes" (1) or "No" (0) as a binary classification. Output json with reason. Here are some examples:
    Example 1
    "question": "What can you tell me about albert Albert Einstein?",
    "context": "Albert Einstein (14 March 1879 - 18 April 1955) was a German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time. Best known for developing the theory of relativity, he also made important contributions to quantum mechanics, and was thus a central figure in the revolutionary reshaping of the scientific understanding of nature that modern physics accomplished in the first decades of the twentieth century. His mass-energy equivalence formula E = mc2, which arises from relativity theory, has been called 'the world's most famous equation'. He received the 1921 Nobel Prize in Physics 'for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect', a pivotal step in the development of quantum theory. His work is also known for its influence on the philosophy of science. In a 1999 poll of 130 leading physicists worldwide by the British journal Physics World, Einstein was ranked the greatest physicist of all time. His intellectual achievements and originality have made Einstein synonymous with genius.",
    "answer": "Albert Einstein born in 14 March 1879 was  German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time. He received the 1921 Nobel Prize in Physics for his services to theoretical physics. He published 4 papers in 1905.  Einstein moved to Switzerland in 1895",
    "classification": {example_1}
    Example 2
    "question": "who won 2020 icc world cup?",
    "context": "The 2022 ICC Men's T20 World Cup, held from October 16 to November 13, 2022, in Australia, was the eighth edition of the tournament. Originally scheduled for 2020, it was postponed due to the COVID-19 pandemic. England emerged victorious, defeating Pakistan by five wickets in the final to clinch their second ICC Men's T20 World Cup title.",
    "answer": "England",
    "classification": {example_2}
    Example 3
    "question": "What is the primary fuel for the Sun?",
    "context": "",
    "answer": "Hydrogen",
    "classification": {example_3}
    <|eot_id|>
    <|start_header_id|>
    "question": {question},
    "context": {context},
    "answer": {answer},<|end_header_id|>
    """
    self.cr_ex1 = json.dumps([
            {
                "statement": "Albert Einstein, born on 14 March 1879, was a German-born theoretical physicist, widely held to be one of the greatest and most influential scientists of all time.",
                "reason": "The date of birth of Einstein is mentioned clearly in the context.",
                "attributed": 1,
            },
            {
                "statement": "He received the 1921 Nobel Prize in Physics for his services to theoretical physics.",
                "reason": "The exact sentence is present in the given context.",
                "attributed": 1,
            },
            {
                "statement": "He published 4 papers in 1905.",
                "reason": "There is no mention about papers he wrote in the given context.",
                "attributed": 0,
            },
            {
                "statement": "Einstein moved to Switzerland in 1895.",
                "reason": "There is no supporting evidence for this in the given context.",
                "attributed": 0,
            },
        ])
    self.cr_ex2 = json.dumps([
            {
                "statement": "England won the 2022 ICC Men's T20 World Cup.",
                "reason": "From context it is clear that England defeated Pakistan to win the World Cup.",
                "attributed": 1,
            },
        ])
    self.cr_ex3 = json.dumps([
            {
                "statement": "The Sun's primary fuel is hydrogen.",
                "reason": "The context contains no information",
                "attributed": 0,
            },
        ])
    self.answer_correctness_prompt = """\
Given a ground truth and an answer statements, analyze each statement and classify them in one of the following categories:

- TP (true positive): statements that are present in answer that are also directly supported by the one or more statements in ground truth,
- FP (false positive): statements present in the answer but not directly supported by any statement in ground truth,
- FN (false negative): statements found in the ground truth but not present in answer.

Each statement can only belong to one of the categories. Provide a reason for each classification. Format the outputs in JSON. You will be punished if you write the code. Here are some examples:
Example 1
"question": "What powers the sun and what is its primary function?",
"answer": [
    "The sun is powered by nuclear fission, similar to nuclear reactors on Earth.",
    "The primary function of the sun is to provide light to the solar system.",
],
"ground_truth": [
    "The sun is powered by nuclear fusion, where hydrogen atoms fuse to form helium.",
    "This fusion process in the sun's core releases a tremendous amount of energy.",
    "The energy from the sun provides heat and light, which are essential for life on Earth.",
    "The sun's light plays a critical role in Earth's climate system.",
    "Sunlight helps to drive the weather and ocean currents.",
],
"classification":
{example_1}
Example 2
"question": "What is the boiling point of water?",
"answer": [
    "The boiling point of water is 100 degrees Celsius at sea level"
],
"ground_truth": [
    "The boiling point of water is 100 degrees Celsius (212 degrees Fahrenheit) at sea level.",
    "The boiling point of water can change with altitude.",
],
"classification":
{example_2}
<|eot_id|>
<|start_header_id|>
"question": {question},
"answer": [
  {answer}
],
"ground_truth": [
  {ground_truth}
],
<|end_header_id|>
"""
    self.ac_ex1 = json.dumps({
    "TP": [
        {
            "statement": "The primary function of the sun is to provide light to the solar system.",
            "reason": "This statement is somewhat supported by the ground truth mentioning the sun providing light and its roles, though it focuses more broadly on the sun's energy.",
        }
    ],
    "FP": [
        {
            "statement": "The sun is powered by nuclear fission, similar to nuclear reactors on Earth.",
            "reason": "This statement is incorrect and contradicts the ground truth which states that the sun is powered by nuclear fusion.",
        }
    ],
    "FN": [
        {
            "statement": "The sun is powered by nuclear fusion, where hydrogen atoms fuse to form helium.",
            "reason": "This accurate description of the sun’s power source is not included in the answer.",
        },
        {
            "statement": "This fusion process in the sun's core releases a tremendous amount of energy.",
            "reason": "This process and its significance are not mentioned in the answer.",
        },
        {
            "statement": "The energy from the sun provides heat and light, which are essential for life on Earth.",
            "reason": "The answer only mentions light, omitting the essential aspects of heat and its necessity for life, which the ground truth covers.",
        },
        {
            "statement": "The sun's light plays a critical role in Earth's climate system.",
            "reason": "This broader impact of the sun’s light on Earth's climate system is not addressed in the answer.",
        },
        {
            "statement": "Sunlight helps to drive the weather and ocean currents.",
            "reason": "The effect of sunlight on weather patterns and ocean currents is omitted in the answer.",
        },
    ],
})
    self.ac_ex2 = json.dumps({
    "TP": [
        {
            "statement": "The boiling point of water is 100 degrees Celsius at sea level",
            "reason": "This statement is directly supported by the ground truth which specifies the boiling point of water as 100 degrees Celsius at sea level.",
        }
    ],
    "FP": [],
    "FN": [
        {
            "statement": "The boiling point of water can change with altitude.",
            "reason": "This additional information about how the boiling point of water can vary with altitude is not mentioned in the answer.",
        }
    ],
})
  def evaluate(self, submit):
    f = self.faithfulness(submit)
    a = self.answer_relevance(submit)
    cp = self.context_precision(submit)
    cr = self.context_recall(submit)
    #ass = self.answer_semantic_similarity(submit)
    #ac = self.answer_correctness(submit)
    ass = None
    ac = None
    result = {"Faithfulness ":f, "Answer relevance":a, "Context precision":cp, "Context recall":cr, "Answer semantic similarity": ass, "Answer correctness": ac}
    return result
  def faithfulness(self, submit):
    score = 0
    passed = 0
    for row in tqdm(submit, desc="Faithfulness evaluation"):
      question = row["question"]
      context = ". ".join(row["context"])
      answer = row["answer"]
      statements_extraction = self.statements_extraction_prompt.format(question=question, answer=answer)
      statements = self.generate(statements_extraction)
      statements = ",\n".join(re.findall("\".*\"", statements))
      statements_verification = self.statements_verification_prompt.format(example=self.verification_example, context=context, statements=statements)
      verification = self.generate(statements_verification)
      verdicts = re.findall("\"verdict\": ([0,1])", verification)
      if len(verdicts) == 0:
        continue
      sample_score = sum([int(ver) for ver in verdicts])/len(verdicts)
      score += sample_score
      passed += 1
    if passed != 0:
      score = score/passed
    return score
  def answer_relevance(self, submit):
    score = 0
    pased = 0
    for row in tqdm(submit, desc="Answer relevance evaluation"):
      question = row["question"]
      context = ". ".join(row["context"])
      answer = row["answer"]
      prompt = self.answer_relevance_prompt.format(answer=answer)
      questions = self.generate(prompt)
      questions = re.findall("\".*\"", questions)
      if len(questions) == 0:
        continue
      embed_gen_questions = self.embed(questions)
      questions = ",\n".join(questions)
      embed_question = self.embed([question])
      local_score = self.cosine_similarity(embed_gen_questions, embed_question)
      score += local_score
      passed += 1
    if passed != 0:
      score = score/passed
    return score
  def context_precision(self, submit):
    score = 0
    passed = 0
    for row in tqdm(submit, desc="Context precision evaluation"):
      question = row["question"]
      context = row["context"]
      answer = row["answer"]
      local_score = 0
      local_passed = 0
      verdict_list = []
      for c in context:
        prompt = self.context_precision_prompt.format(question=question, context=c, answer=answer, example_1=self.cp_ex1, example_2=self.cp_ex2, example_3=self.cp_ex3)
        verification = self.generate(prompt)
        verdicts = re.findall("\"verdict\": ([0,1])", verification)
        if len(verdicts) == 0:
          continue
        verdict_list.append(int(verdicts[0]))
        local_passed += 1
      if local_passed == 0:
        continue
      denominator = sum(verdict_list)
      if denominator == 0:
        continue
      numerator = sum(
          [
              (sum(verdict_list[: i + 1]) / (i + 1)) * verdict_list[i]
              for i in range(len(verdict_list))
          ]
      )
      precision = numerator / denominator
      score += precision
      passed += 1
    if passed != 0:
      score = score/passed
    return score
  def context_recall(self, submit):
    score = 0
    passed = 0
    for row in tqdm(submit, desc="Context recall evaluation"):
      question = row["question"]
      context = ". ".join(row["context"])
      answer = row["answer"]
      prompt = self.context_recall_prompt.format(question=question, context=context, answer=answer, example_1=self.cr_ex1, example_2=self.cr_ex2, example_3=self.cr_ex3)
      verification = self.generate(prompt)
      verdicts = re.findall("\"attributed\": ([0,1])", verification)
      if len(verdicts) == 0:
        continue
      sample_score = sum([int(ver) for ver in verdicts])/len(verdicts)
      score += sample_score
      passed += 1
    if passed != 0:
      score = score/passed
    return score
  def answer_semantic_similarity(self, submit):
    print("Answer semantic similarity evaluation")
    answer = submit["answer"]
    gt = submit["ground_truths"]
    answers_embed = self.embed(answer)
    gt_embed = self.embed(gt)
    score = self.cosine_similarity(answers_embed, gt_embed)
    return score
  def answer_correctness(self, submit):
    score = 0
    passed = 0
    for row in tqdm(submit, desc="Answer correctness evaluation"):
      question = row["question"]
      context = ". ".join(row["context"])
      answer = row["answer"]
      gt = row["ground_truths"]
      gen_statements_extraction = self.statements_extraction_prompt.format(question=question, answer=answer)
      gen_statements = self.generate(gen_statements_extraction)
      gt_statements_extraction = self.statements_extraction_prompt.format(question=question, answer=gt)
      gt_statements = self.generate(gt_statements_extraction)
      gen_statements = ",\n".join(re.findall("\".*\"", gen_statements))
      gt_statements = ",\n".join(re.findall("\".*\"", gt_statements))
      ac_prompt = self.answer_correctness_prompt.format(question=question, answer=gen_statements, ground_truth=gt_statements, example_1=self.ac_ex1, example_2=self.ac_ex2)
      classification = self.generate(ac_prompt)
      tp = re.findall(""""TP": \[(.*?)\]""", classification)
      fp = re.findall(""""FP": \[(.*?)\]""", classification)
      fn = re.findall(""""FN": \[(.*?)\]""", classification)
      if len(tp) == 0 or len(fp) == 0 or len(fn) == 0:
        continue
      tp = len(re.findall("\{.*?\}", tp[0]))
      fp = len(re.findall("\{.*?\}", fp[0]))
      fn = len(re.findall("\{.*?\}", fn[0]))
      local_score = tp / (tp + 0.5 * (fp + fn)) if tp > 0 else 0
      score += local_score
      passed += 1
    score = score/passed
    return score

  def generate(self, prompt):
    tokenized_prompt = self.judge_tokenizer(prompt, return_tensors='pt')['input_ids'].cuda()
    gen_config = GenerationConfig.from_pretrained(self.judge_model.name_or_path)
    gen_config.max_length = 4096
    gen_config.pad_token_id = self.judge_tokenizer.eos_token_id
    with torch.inference_mode():
        output = self.judge_model.generate(**{"input_ids":tokenized_prompt, "generation_config":gen_config})
    output = self.judge_tokenizer.decode(output[0][len(tokenized_prompt[0]):], skip_special_tokens=True)
    return output
  def mean_pooling(self, model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask
  def embed(self, sentences):
    encoded_input = self.embedder_tokenizer(sentences, padding=True, truncation=True, max_length=128, return_tensors='pt')
    with torch.no_grad():
        model_output = self.embedder_model(**encoded_input)
    sentence_embeddings = self.mean_pooling(model_output, encoded_input['attention_mask'])
    return sentence_embeddings
  def cosine_similarity(self, candidate_embs, comparing_emb):
    if comparing_emb.shape[0] == 1:
      comparing_emb = comparing_emb.repeat((candidate_embs.shape[0], 1))
    cos = nn.CosineSimilarity()
    sim = cos(candidate_embs, comparing_emb)
    sim = sim.sum()/candidate_embs.shape[0]
    return sim.item()

In [33]:
def fix_dataset(row):
  row["context"] = row['wiki_context']
  return row

In [34]:
ragas_params = {
    "judge_name":"VityaVitalich/Llama3.1-8b-instruct",
    "embedder_name":"ai-forever/sbert_large_mt_nlu_ru"}

In [35]:
ragas = RAGAS(ragas_params)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [62]:
pipelines = {
    'rag_answer_quat': answer_with_rag_quat,
    'rag_answer': answer_with_rag,
}
scores = {
    'rag_answer_quat': 0,
    'rag_answer': 0,
    'llm_answer': 0
}

In [41]:
ds_test = ds_test.rename_column("answer", "ground_truth")
ds_test = ds_test.map(fix_dataset)

Map:   0%|          | 0/7701 [00:00<?, ? examples/s]

In [48]:
from huggingface_hub import login
login()

In [63]:
import gc
for key in pipelines.keys():
  pipe = pipelines[key]
  if key != "llm_answer":
    answers = []
    for row in tqdm(dataset):
      ans = pipe(row['question'], READER_LLM, KNOWLEDGE_VECTOR_DATABASE, reranker=RERANKER)[0]
      answers.append(ans)
      del ans
      gc.collect()
      torch.cuda.empty_cache()
    dataset = dataset.add_column(key, answers)
    dataset.push_to_hub("evgmaslov/rag")
    #dataset = dataset.map(lambda x: {'answer': pipe(x['question'], READER_LLM, KNOWLEDGE_VECTOR_DATABASE, reranker=RERANKER)[0]})
  else:
    answers = []
    for row in tqdm(dataset):
      ans = pipe(row['question'])
      answers.append(ans)
      del ans
      gc.collect()
      torch.cuda.empty_cache()
    dataset = dataset.add_column(key, answers)
    dataset.push_to_hub("evgmaslov/rag")
    #dataset = dataset.map(lambda x: {'answer': pipe(x['question'])})

  0%|          | 0/50 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00,  4.21it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

100%|██████████| 1/1 [00:00<00:00,  3.59it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

100%|██████████| 1/1 [00:00<00:00,  4.00it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

100%|██████████| 1/1 [00:00<00:00,  3.39it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

100%|██████████| 1/1 [00:00<00:00,  3.67it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

100%|██████████| 1/1 [00:00<00:00,  3.70it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

100%|██████████| 1/1 [00:00<00:00,  3.76it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

100%|██████████| 1/1 [00:00<00:00,  4.00it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

100%|██████████| 1/1 [00:00<00:00,  3.7

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

  0%|          | 0/50 [00:00<?, ?it/s]


100%|██████████| 1/1 [00:00<00:00,  3.78it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

100%|██████████| 1/1 [00:00<00:00,  3.62it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

100%|██████████| 1/1 [00:00<00:00,  3.89it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

100%|██████████| 1/1 [00:00<00:00,  3.79it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

100%|██████████| 1/1 [00:00<00:00,  4.12it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

100%|██████████| 1/1 [00:00<00:00,  3.39it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

100%|██████████| 1/1 [00:00<00:00,  3.54it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

100%|██████████| 1/1 [00:00<00:00,  3.79it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.

100%|██████████| 1/1 [00:00<00:00,  3.9

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

In [ ]:
print(scores)